In [3]:
import pandas as pd
from selenium import webdriver
import time
from bs4 import BeautifulSoup

# 启动浏览器（可设置为无头）
driver = webdriver.Edge()

# 访问目标网站
url = 'https://data.eastmoney.com/zjlx/list.html'
driver.get(url)

# 等待某个关键元素出现，说明数据已加载（例如等“涨幅”字段不再是“-”）
time.sleep(5)

# 提取表格 HTML
html = driver.page_source

# 用 BeautifulSoup 解析 HTML
soup = BeautifulSoup(html, "html.parser")

# 找到 class 为 dataview-body 的容器
container = soup.find("div", class_="dataview-body")

# 在其中找 table 标签
table = container.find("table")

stock_data = {}

for row in table.find_all('tr')[2:]:  # 前两行为表头
    cells = row.find_all('td')
    if len(cells) < 15:
        continue
    
    stock_code = cells[1].get_text(strip=True)


    for cell in cells:
        a = cell.find('a')
        if a and a.get('href'):
            data = {}
            text = a.get_text(strip=True)
            href = a['href']
            if text == '详情':
                data[text] = "https://data.eastmoney.com" + href       
                break  

    stock_data[stock_code] = {
        "序号": cells[0].get_text(strip=True),
        "代码": stock_code,
        "名称": cells[2].get_text(strip=True),
        "相关资讯": data,
        "最新价": cells[4].get_text(strip=True),
        "今日主力净占比": cells[5].get_text(strip=True),
        "今日排名": cells[6].get_text(strip=True),
        "今日涨跌": cells[7].get_text(strip=True),
        "5日主力净占比": cells[8].get_text(strip=True),
        "5日排名": cells[9].get_text(strip=True),
        "5日涨跌": cells[10].get_text(strip=True),
        "10日主力净占比": cells[11].get_text(strip=True),
        "10日排名": cells[12].get_text(strip=True),
        "10日涨跌": cells[13].get_text(strip=True),
        "所属板块": cells[14].get_text(strip=True),
    }

# 打印结果查看
from pprint import pprint
pprint(stock_data)

{'000025': {'10日主力净占比': '0.61%',
            '10日排名': '932',
            '10日涨跌': '-2.18%',
            '5日主力净占比': '5.75%',
            '5日排名': '246',
            '5日涨跌': '14.74%',
            '今日主力净占比': '31.81%',
            '今日排名': '15',
            '今日涨跌': '9.99%',
            '代码': '000025',
            '名称': '特  力Ａ',
            '序号': '15',
            '所属板块': '珠宝首饰',
            '最新价': '17.51',
            '相关资讯': {'详情': 'https://data.eastmoney.com/zjlx/000025.html'}},
 '000402': {'10日主力净占比': '0.42%',
            '10日排名': '1017',
            '10日涨跌': '-3.21%',
            '5日主力净占比': '4.74%',
            '5日排名': '369',
            '5日涨跌': '0.00%',
            '今日主力净占比': '19.30%',
            '今日排名': '42',
            '今日涨跌': '1.12%',
            '代码': '000402',
            '名称': '金 融 街',
            '序号': '41',
            '所属板块': '房地产开发',
            '最新价': '2.71',
            '相关资讯': {'详情': 'https://data.eastmoney.com/zjlx/000402.html'}},
 '000425': {'10日主力净占比': '3.58%',
       

In [4]:
# 使用 pd.read_html 来提取每个详情链接的表格
all_details_data = []

for stock_code, stock_info in stock_data.items():
    # 获取详情链接
    detail_url = stock_info["相关资讯"].get('详情', None)
    if detail_url:
        # 使用 Selenium 打开详情页
        driver.get(detail_url)
        time.sleep(3)  # 等待页面加载完成
        
        # 使用 pd.read_html 从页面提取所有表格数据
        try:
            dfs = pd.read_html(driver.page_source)  # 返回一个包含所有表格的 DataFrame 列表
            if dfs:
                # 第四个表格是我们需要的数据，选择第四个表格
                details_df = dfs[3]
                details_df['股票代码'] = stock_code  # 添加股票代码
                all_details_data.append(details_df)
        except Exception as e:
            print(f"Error extracting table for stock {stock_code}: {e}")

# 合并所有详情数据
if all_details_data:
    final_details_df = pd.concat(all_details_data, ignore_index=True)
    print(final_details_df)

    # 保存为 CSV 文件
    final_details_df.to_csv("stock_details_data.csv", encoding="utf-8-sig", index=False)

# 关闭浏览器
driver.quit()

C:\Users\m1830\AppData\Local\Temp\ipykernel_29948\2610049326.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(driver.page_source)  # 返回一个包含所有表格的 DataFrame 列表
C:\Users\m1830\AppData\Local\Temp\ipykernel_29948\2610049326.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(driver.page_source)  # 返回一个包含所有表格的 DataFrame 列表
C:\Users\m1830\AppData\Local\Temp\ipykernel_29948\2610049326.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(driver.page_source)  # 返回一个包含所有表格的 DataFrame 列表
C:\Users\m1830\AppData\Local\Temp\ipykernel_29948\2610049326.py:14: FutureWarning:

              日期   收盘价     涨跌幅     主力净流入          超大单净流入             大单净流入  \
              日期   收盘价     涨跌幅        净额     净占比      净额     净占比        净额   
0     2025-04-16  2.17   0.00%    34.99万  61.45%    0.00   0.00%    34.99万   
1     2025-04-15  2.17   0.00%      0.00   0.00%    0.00   0.00%      0.00   
2     2025-04-14  2.17  -0.91%      0.00   0.00%    0.00   0.00%      0.00   
3     2025-04-11  2.19  -0.90%      0.00   0.00%    0.00   0.00%      0.00   
4     2025-04-10  2.21   0.45%      0.00   0.00%    0.00   0.00%      0.00   
...          ...   ...     ...       ...     ...     ...     ...       ...   
5841  2024-10-24  9.19  -0.22%    46.33万   7.65%    0.00   0.00%    46.33万   
5842  2024-10-23  9.21  -0.32%   -46.34万  -4.07%    0.00   0.00%   -46.34万   
5843  2024-10-22  9.24   0.22%  -142.32万  -9.21%  13.94万   0.90%  -156.26万   
5844  2024-10-21  9.22   0.44%  -115.81万  -6.63%  -1.75万  -0.10%  -114.06万   
5845  2024-10-18  9.18   1.55%   -81.91万  -4.30%    0.00   0.00%

In [5]:
# 保存为 CSV 文件
final_details_df.to_csv("stock_details_data.csv", encoding="utf-8-sig", index=False)


In [7]:
final_details_df[final_details_df['股票代码']=='000425'].to_csv('000425.csv')

In [8]:
import requests
import pandas as pd
import time
import math
from fake_useragent import UserAgent

ua = UserAgent()

def get_stock_data():
    all_data = []

    base_url = "https://push2.eastmoney.com/api/qt/clist/get"
    params = {
        "fid": "f184",
        "po": "1",
        "pz": "50",  # 每页数量
        "pn": "1",   # 当前页
        "np": "1",
        "fltt": "2",
        "invt": "2",
        "ut": "8dec03ba335b81bf4ebdf7b29ec27d15",
        "fs": "m:0+t:6+f:!2,m:0+t:13+f:!2,m:0+t:80+f:!2,m:1+t:2+f:!2,m:1+t:23+f:!2,m:0+t:7+f:!2,m:1+t:3+f:!2",
        "fields": "f2,f3,f12,f14,f62,f184,f225,f109,f160,f124,f100,f1"
    }

    headers = {
        "User-Agent": ua.random,
        "Referer": "https://data.eastmoney.com/zjlx/list.html"
    }

    # 先请求一页来获取总条数
    res = requests.get(base_url, params=params, headers=headers)
    res_json = res.json()
    total = res_json['data']['total']
    pages = math.ceil(total / int(params['pz']))

    print(f"共找到 {total} 条数据，分为 {pages} 页")

    # 遍历每一页
    for page in range(1, pages + 1):
        print(f"正在抓取第 {page} 页...")
        params['pn'] = str(page)
        res = requests.get(base_url, params=params, headers=headers)
        data = res.json()['data']['diff']
        all_data.extend(data)
        time.sleep(0.5)  # 防止被封，稍微休息一下

    df = pd.DataFrame(all_data)
    return df


if __name__ == '__main__':
    df = get_stock_data()
    print(df.head())
    df.to_csv("资金流.csv", index=False, encoding='utf-8-sig')


共找到 5232 条数据，分为 105 页
正在抓取第 1 页...
正在抓取第 2 页...
正在抓取第 3 页...
正在抓取第 4 页...
正在抓取第 5 页...
正在抓取第 6 页...
正在抓取第 7 页...
正在抓取第 8 页...
正在抓取第 9 页...
正在抓取第 10 页...
正在抓取第 11 页...
正在抓取第 12 页...
正在抓取第 13 页...
正在抓取第 14 页...
正在抓取第 15 页...
正在抓取第 16 页...
正在抓取第 17 页...
正在抓取第 18 页...
正在抓取第 19 页...
正在抓取第 20 页...
正在抓取第 21 页...
正在抓取第 22 页...
正在抓取第 23 页...
正在抓取第 24 页...
正在抓取第 25 页...
正在抓取第 26 页...
正在抓取第 27 页...
正在抓取第 28 页...
正在抓取第 29 页...
正在抓取第 30 页...
正在抓取第 31 页...
正在抓取第 32 页...
正在抓取第 33 页...
正在抓取第 34 页...
正在抓取第 35 页...
正在抓取第 36 页...
正在抓取第 37 页...
正在抓取第 38 页...
正在抓取第 39 页...
正在抓取第 40 页...
正在抓取第 41 页...
正在抓取第 42 页...
正在抓取第 43 页...
正在抓取第 44 页...
正在抓取第 45 页...
正在抓取第 46 页...
正在抓取第 47 页...
正在抓取第 48 页...
正在抓取第 49 页...
正在抓取第 50 页...
正在抓取第 51 页...
正在抓取第 52 页...
正在抓取第 53 页...
正在抓取第 54 页...
正在抓取第 55 页...
正在抓取第 56 页...
正在抓取第 57 页...
正在抓取第 58 页...
正在抓取第 59 页...
正在抓取第 60 页...
正在抓取第 61 页...
正在抓取第 62 页...
正在抓取第 63 页...
正在抓取第 64 页...
正在抓取第 65 页...
正在抓取第 66 页...
正在抓取第 67 页...
正在抓取第 68 页...
正在抓取第 69 页...
正在抓取第 70 页...
正在抓取第 7

In [19]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from fake_useragent import UserAgent
from urllib.parse import urljoin
import time
from selenium import webdriver
import math

class FundFlowScraper:
    def __init__(self, page_size=50, sleep_sec=0.5):
        self.ua = UserAgent()
        self.page_size = page_size
        self.sleep_sec = sleep_sec
        self.base_url = "https://push2.eastmoney.com/api/qt/clist/get"
        self.params = {
            "fid": "f184",
            "po": "1",
            "pz": str(page_size),
            "pn": "1",
            "np": "1",
            "fltt": "2",
            "invt": "2",
            "ut": "8dec03ba335b81bf4ebdf7b29ec27d15",
            "fs": "m:0+t:6+f:!2,m:0+t:13+f:!2,m:0+t:80+f:!2,m:1+t:2+f:!2,m:1+t:23+f:!2,m:0+t:7+f:!2,m:1+t:3+f:!2",
            "fields": "f2,f3,f12,f14,f100,f109,f160,f165,f175,f184,f225,f263,f264"
        }
        self.headers = {
            "User-Agent": self.ua.random,
            "Referer": "https://data.eastmoney.com/zjlx/list.html"
        }
        self.field_map = {
            "f2": "最新价",
            "f3": "今日涨跌(%)",
            "f225": "今日排名",
            "f12": "代码",
            "f14": "名称",
            "f100": "所属板块",
            "f109": "5日涨跌",
            "f160": "10日涨跌",
            "f165": "5日排行榜主力净占比",
            "f175": "10日排行榜主力净占比",
            "f184": "今日排行榜主力净占比",
            "f263": "5日排名",
            "f264": "10日排名"
        }


    def _get_total_pages(self):
        res = requests.get(self.base_url, params=self.params, headers=self.headers)
        res_json = res.json()
        total = res_json['data']['total']
        pages = math.ceil(total / self.page_size)
        return total, pages

    def scrape_all(self):
        total, pages = self._get_total_pages()
        print(f"共 {total} 条数据，{pages} 页，每页 {self.page_size} 条")
        all_data = []

        for page in range(1, pages + 1):
            print(f"抓取第 {page} 页...")
            self.params['pn'] = str(page)
            res = requests.get(self.base_url, params=self.params, headers=self.headers)
            res_json = res.json()
            data = res_json['data']['diff']
            all_data.extend(data)
            time.sleep(self.sleep_sec)

        df = pd.DataFrame(all_data)
        df = df.rename(columns=self.field_map)
        return df

    def save_csv(self, df: pd.DataFrame, filename="资金流.csv"):
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"数据已保存到 {filename}")

scraper = FundFlowScraper(page_size=50)
df = scraper.scrape_all()


共 5232 条数据，105 页，每页 50 条
抓取第 1 页...
抓取第 2 页...
抓取第 3 页...
抓取第 4 页...
抓取第 5 页...
抓取第 6 页...
抓取第 7 页...
抓取第 8 页...
抓取第 9 页...
抓取第 10 页...
抓取第 11 页...
抓取第 12 页...
抓取第 13 页...
抓取第 14 页...
抓取第 15 页...
抓取第 16 页...
抓取第 17 页...
抓取第 18 页...
抓取第 19 页...
抓取第 20 页...
抓取第 21 页...
抓取第 22 页...
抓取第 23 页...
抓取第 24 页...
抓取第 25 页...
抓取第 26 页...
抓取第 27 页...
抓取第 28 页...
抓取第 29 页...
抓取第 30 页...
抓取第 31 页...
抓取第 32 页...
抓取第 33 页...
抓取第 34 页...
抓取第 35 页...
抓取第 36 页...
抓取第 37 页...
抓取第 38 页...
抓取第 39 页...
抓取第 40 页...
抓取第 41 页...
抓取第 42 页...
抓取第 43 页...
抓取第 44 页...
抓取第 45 页...
抓取第 46 页...
抓取第 47 页...
抓取第 48 页...
抓取第 49 页...
抓取第 50 页...
抓取第 51 页...
抓取第 52 页...
抓取第 53 页...
抓取第 54 页...
抓取第 55 页...
抓取第 56 页...
抓取第 57 页...
抓取第 58 页...
抓取第 59 页...
抓取第 60 页...
抓取第 61 页...
抓取第 62 页...
抓取第 63 页...
抓取第 64 页...
抓取第 65 页...
抓取第 66 页...
抓取第 67 页...
抓取第 68 页...
抓取第 69 页...
抓取第 70 页...
抓取第 71 页...
抓取第 72 页...
抓取第 73 页...
抓取第 74 页...
抓取第 75 页...
抓取第 76 页...
抓取第 77 页...
抓取第 78 页...
抓取第 79 页...
抓取第 80 页...
抓取第 81 页...
抓取第 82 页...


In [ ]:
field_mapping_zh = {
    "f2": "最新价",
    "f3": "今日涨跌(%)",
    "f225": "今日排名",
    "f12": "代码",
    "f14": "名称",
    "f100": "所属板块",
    "f109": "5日涨跌",
    "f160": "10日涨跌",
    "f165": "5日排行榜主力净占比",
    "f175": "10日排行榜主力净占比",
    "f184": "今日排行榜主力净占比",
    "f263": "5日排名",
    "f264": "10日排名"
}


In [20]:
df.head()

,最新价,今日涨跌(%),代码,名称,所属板块,5日涨跌,10日涨跌,5日排行榜主力净占比,10日排行榜主力净占比,今日排行榜主力净占比,今日排名,5日排名,10日排名
0,2.17,0.0,200992,中 鲁Ｂ,农牧饲渔,-1.36,-5.24,19.4,4.96,61.45,1,3,92
1,7.43,10.07,002040,南 京 港,航运港口,7.22,11.90,-0.1,3.0,59.23,2,2094,270
2,37.69,10.01,603205,健尔康,医疗器械,15.26,-0.50,19.85,7.52,41.26,3,2,32
3,6.07,9.96,002909,集泰股份,化学制品,20.20,4.48,13.91,5.49,39.0,4,19,74
4,25.55,9.99,001234,泰慕士,纺织服装,61.10,40.00,2.0,1.54,38.16,5,1031,586


In [21]:
df.to_csv('data.csv',encoding='gbk')

In [ ]:
import requests
import pandas as pd
import time

class EastmoneyKlineFetcher:
    def __init__(self, stock_list, klt="101", fqt="1", lmt=120):
        self.stock_list = stock_list  # ['0.002040', '1.600519']
        self.klt = klt
        self.fqt = fqt
        self.lmt = lmt

    def fetch_single(self, secid):
        url = (
            f"https://push2his.eastmoney.com/api/qt/stock/kline/get?"
            f"secid={secid}&ut=fa5fd1943c7b386f172d6893dbfba10b"
            f"&fields1=f1,f2,f3,f4,f5,f6"
            f"&fields2=f51,f52,f53,f54,f55,f56,f57,f58,f59,f60"
            f"&klt={self.klt}&fqt={self.fqt}&end=20500101&lmt={self.lmt}"
        )
        try:
            response = requests.get(url, timeout=10)
            json_data = response.json()
            if not json_data.get("data"):
                return None
            klines = json_data["data"]["klines"]
            df = pd.DataFrame([line.split(',') for line in klines], columns=[
                "日期", "开盘", "收盘", "最高", "最低", "成交量", "成交额", "振幅", "涨跌额", "涨跌幅"
            ])
            df["股票代码"] = secid
            return df
        except Exception as e:
            print(f"Error fetching {secid}: {e}")
            return None

    def fetch_all(self):
        all_data = []
        for secid in self.stock_list:
            print(f"Fetching: {secid}")
            df = self.fetch_single(secid)
            if df is not None:
                all_data.append(df)
            time.sleep(0.5)  # 防止被限速
        return pd.concat(all_data, ignore_index=True) if all_data else None
